In [1]:
import sys
sys.path.insert(0, '../../src')
import feedparser
import pandas as pd

import urllib

from postagem.Util import extract_domain, download_and_move_image, get_noticia_comercio
from lexical_analyzer_package import midia_lexical
from midia_postagem import midia_post
from Model.News import News
from Database import midia_table

from newsplease import NewsPlease
from bs4 import BeautifulSoup
import requests

import datetime

In [2]:
link = 'http://blogdoriella.com.br/'

In [3]:
req = requests.get(link)

In [4]:
noticias = BeautifulSoup(req.text, "html.parser").find_all('h2', class_='post-title entry-title')

In [5]:
for noticia in noticias:
    print(noticia.find_all('a', href=True)[0]['href'])
    article = NewsPlease.from_url(noticia.find_all('a', href=True)[0]['href'])
    row = {'titulos': [], 'links': [], 'noticia': [], 'image': [], 'abstract': [], 'date': []}
    if (article is not None):
        row['titulos'].append(article.title)
        row['noticia'].append(article.text)
        row['links'].append(article.url)
        row['abstract'].append(article.text)
        row['date'].append(article.date_publish)
        path_image = article.image_url
        print(path_image)
        if path_image == '' or path_image == None:
            row['image'].append(0)
        else:
            row['image'].append(download_and_move_image(article.image_url))
        news = News(row['abstract'], row['noticia'], row['date'], row['links'], row['titulos'], row['image'])
        try:
            print(row['titulos'])
            news_in_db = midia_table.check_news(news)
            print('news_in_db: ' + str(news_in_db))
            if (not news_in_db):
                row = pd.DataFrame(row)
                df, categories = midia_lexical.lexical_corpus_and_title(row)
                # DB categories
                if (categories != [set()]):
                    news.set_categories(categories)
                    midia_table.save_news(news)
                    midia_post.post_news(df)
        except:
            print('Empty News')

http://blogdoriella.com.br/brasil-tem-mais-de-19-mil-casos-de-sarampo-confirmados/
http://blogdoriella.com.br/wp-content/uploads/2014/10/miniatura.png
HTTP Error 404: Not Found
['Brasil tem mais de 1,9 mil casos de sarampo confirmados']
news_in_db: False
 -- no categories -- 
http://blogdoriella.com.br/eua-anunciam-fim-de-tratado-de-amizade-firmado-em-1955-com-ira/
http://blogdoriella.com.br/wp-content/uploads/2014/10/miniatura.png
HTTP Error 404: Not Found
['EUA anunciam fim de tratado de amizade firmado em 1955 com Irã']
news_in_db: False
 -- no categories -- 
http://blogdoriella.com.br/ibge-pais-perdeu-708-mil-empresas-em-2016-e-dispensou-de-16-milhao-de-trabalhadores/
http://blogdoriella.com.br/wp-content/uploads/2014/10/miniatura.png
HTTP Error 404: Not Found
['IBGE: país perdeu 70,8 mil empresas em 2016 e dispensou de 1,6 milhão de trabalhadores']
news_in_db: False
 -- no categories -- 
http://blogdoriella.com.br/medicos-barram-participacao-de-candidato-em-ultimo-debate-na-tv/
ht

KeyboardInterrupt: 

In [ ]:
noticias[0]